## Imports for the Project and Setup

In [ ]:
# Full reset of the camera
!echo 'dlinano' | sudo -S systemctl restart nvargus-daemon && printf '\n'
# Check device number
#!ls -ltrh /dev/video*

In [2]:
import torch
import torchvision
#from torchvision import transforms , datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary # for model summary
import matplotlib.pyplot as plt # for ploting our data and showing images
import matplotlib.image as mpimg
import numpy as np
import cv2
import dataHelper as dH
import os
import time
import random
import ipywidgets
from IPython.display import display
%matplotlib inline

In [2]:
from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera
camera = CSICamera(width=224, height=224)

## Model Setup

Here we are letting pytorch know to use the gpu and we are also importing a pretrained resnet18 model that we are modifying to return only 2 outputs. One for X and one for Y.

In [5]:
device = torch.device('cuda')
output_dim = 2
# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)
model = model.to(device)

## Model Summary
This is the inner workings of resnet18.

In [ ]:
summary(model, input_size=(3,224,224))

## Data Collection

### Files Structure Setup

In [6]:
path_raw = os.getcwd() + '/raw_datasets'
try:
    os.mkdir(path_raw)
except OSError:
    pass
else:
    pass

In [ ]:
#config
datacollect = False
dataSetSize = 150
FPScollect = 30
SEC_delay = .5


loopcounter = 0
#collectedNum = 0
collectedNum = 75

if datacollect == True:
    image_widget = ipywidgets.Image(format='jpeg')
    captured_frame = ipywidgets.Image(format='jpeg')
    
    camera.unobserve_all()
    print('Get Ready!')
    frame = camera.read()
    image_widget.value = bgr8_to_jpeg(frame)
    captured_frame.value = bgr8_to_jpeg(frame)
    #plt.imshow(bgr8_to_jpeg(frame))
    time.sleep(6)
    display(ipywidgets.HBox([image_widget, captured_frame]))
    print('Begin!')
    
    while True:
        if collectedNum == dataSetSize:
            break
        frame = camera.read()
        #plt.imshow(frame)
        image_widget.value = bgr8_to_jpeg(frame)
        if loopcounter % (FPScollect * SEC_delay) == 0:
            # write raw images to file 
            captured_frame.value = bgr8_to_jpeg(frame)
            dH.write_raw_frame(path_raw, frame, collectedNum)
            collectedNum += 1
        loopcounter += 1
print('Done!')

Get Ready!


Begin!


## Data Labeling

An I oop... data is labeled on aonther PC because some libraries are broken

## Training Data Build

In [4]:

REBUILD = False


path_labeled = os.getcwd() + '/labeled_datasets'
data = os.scandir(path_labeled)


#fig = plt.figure()
#ax = fig.add_subplot(1,1,1)
#plt.axis("off")

fill = np.zeros((224,224,2))

if REBUILD:
    training_data = []
    for i in data:
        if '.jpg' in i.name:
            img = mpimg.imread(path_labeled +'/'+ i.name)
            [imgnum, x, y] = dH.label_parser(path_labeled +'/'+ i.name)
            #print(f'{imgnum},{x},{y}')
            #imgplot = plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            training_data.append([np.array(img),np.array([x,y])])
    np.random.shuffle(training_data)
    np.save('training_data.npy', training_data)



## Model Training

In [9]:
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
loss_func = nn.MSELoss()

TRAIN = True

EPOCHS = 500
BATCH_SIZE = 10

if False:
    model = torch.load('test_run_1')

if TRAIN:
    training_data = np.load('training_data.npy', allow_pickle=True)
    
    training_img = torch.Tensor([i[0] for i in training_data]).view(-1,3,224,224).to(device)
    training_label = torch.Tensor([i[1] for i in training_data]).view(-1,2).to(device)
    
    for epoch in range(EPOCHS):
        # randomize each epoch
        rand_idx = torch.randperm(len(training_data))
        
        for i in range(0, len(training_data), BATCH_SIZE):
            model.zero_grad()
            
            output = model(training_img[rand_idx[i:i+BATCH_SIZE]].view(-1,3,224,224))
            computed_output = 224 * (output / 2.0 + 0.5)
            
            loss = loss_func(computed_output,training_label[rand_idx[i:i+BATCH_SIZE]].view(-1,2))
            loss.backward()
            optimizer.step()
            #break
        #break
        print(loss)
    '''
    print('testing')
    print(output)
    print(computed_output)
    #plt.imshow(check.cpu().view(224,224,3).numpy().astype(int))
    '''

tensor(291.1420, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(438.0991, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1948.7723, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(566.9511, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(945.6880, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(127.1462, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(75.8670, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(128.5431, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(378.0761, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(507.6480, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(203.0817, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(411.2409, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1437.2476, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(692.5820, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(453.7930, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(813.7488, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(146.7449, device

In [10]:
torch.save(model, 'test_run_1')

In [ ]:
lab=training_label[rand_idx[0:8]]
lab

In [16]:
test_idx = 8
o = model(training_img[test_idx].view(-1,3,224,224))
c_output = 224 * (o / 2.0 + 0.5)
L = training_label[test_idx]
print(c_output)
print(L)

tensor([[63.0307, 49.8782]], device='cuda:0', grad_fn=<MulBackward0>)
tensor([49., 28.], device='cuda:0')


## Evaluation

In [ ]:

Eval = False

if Eval:
    camera.unobserve_all()
    eval_widget = ipywidgets.Image(format='jpeg')
    